In [1]:
import re
import nltk
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
from pickle import dump, load
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# 文件路径列表
file_paths = [
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_1.jsonl',
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_2.jsonl',
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_3.jsonl',
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_4.jsonl',
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_5.jsonl'
]

# 打开一个新文件用于写入所有标题
with open('all_titles.txt', 'w', encoding='utf-8') as out_file:
    # 遍历文件路径
    for file_path in file_paths:
        # 打开文件并逐行读取
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # 将每行的内容从JSON字符串转换为字典
                article = json.loads(line)
                # 替换掉标题中的换行符'\n'，然后写入到输出文件中
                title = article['title'].replace('\n', ' ')
                out_file.write(title + '\n')


In [ ]:
def preprocess_text(text):
    """
    预处理文本：转换为小写并去除非字母数字字符
    """
    # 转换为小写
    text = text.lower()
    # 使用正则表达式去除非字母数字字符
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

# 输入文件路径
input_file_path = 'all_titles.txt'
# 输出文件路径
output_file_path = 'preprocessed_titles.txt'

# 读取输入文件，预处理每一行，然后写入到输出文件
with open(input_file_path, 'r', encoding='utf-8') as in_file, \
     open(output_file_path, 'w', encoding='utf-8') as out_file:
    for line in in_file:
        preprocessed_line = preprocess_text(line)
        out_file.write(preprocessed_line + '\n')


In [5]:
# 确保已下载nltk的words资源
nltk.download('words', quiet=True)

from nltk.corpus import words

def preprocess_titles(file_path):
    # 获取英文词典中的单词列表
    english_words = set(words.words())
    processed_titles = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 移除数字并去除首尾空格
            line_without_numbers = re.sub(r'\d+', '', line).strip()
            # 转换为小写进行比较
            line_lower = line_without_numbers.lower()
            # 检查是否为不需要的标题，考虑首尾空格和大小写
            if line_lower == "no title found":
                continue
            # 分词
            words_in_title = line_without_numbers.split()
            # 检查每个单词是否在英文词典中，同时确保标题长度大于1
            if all(word.lower() in english_words for word in words_in_title) and len(words_in_title) > 1:
                processed_titles.append(line_without_numbers)
    
    return processed_titles

def save_processed_titles(processed_titles, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for title in processed_titles:
            file.write(title + '\n')

# 文件路径
file_path = 'preprocessed_titles.txt'
output_file_path = 'processed_titles_final.txt'

processed_titles = preprocess_titles(file_path)
save_processed_titles(processed_titles, output_file_path)

print(f"Final processed titles, excluding 'no title found', have been saved to {output_file_path}.")


Final processed titles, excluding 'no title found', have been saved to processed_titles_final.txt.


In [6]:
file_path = 'processed_titles_final.txt'

titles = []
with open(file_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i < 5000:  # 仅读取前5000行
            title = line.strip().lower()
            titles.append(title)
        else:
            break  # 当读取到5000行时，退出循环

In [7]:
titles

['the right formula',
 'the nature buffer the missing link in climate change and mental health research',
 'on the approximate compressibility of connected vertex cover',
 'man and insect past present future',
 'doc will my relative wake up and be normal',
 'the role and future of the pathologist in cytology',
 'the effects of deep pressure touch on anxiety',
 'in this issue',
 'significance of the gravitation factor in the final stage of a space flight',
 'splenic abscess a new case',
 'field trial of talc direct recording scale',
 'the view box',
 'calling a spade a spade mathematics in the new pattern of division of   labour',
 'quadrupole association and dissociation of hydrogen in the early   universe',
 'intrapericardial teratoma',
 'the sketch of a polymorphic symphony',
 'embryonal rhabdomyosarcoma associated with maternal ingestion of stilboestrol',
 'capsule routing for sound event detection',
 'the photographer in the operating room',
 'the microwave resonance therapy of ref

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(titles)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in titles:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# 填充序列以保持统一长度
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


# 分割为特征和标签
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = to_categorical(label, num_classes=total_words)

# Model V2

In [21]:
# 定义模型
def create_model(total_words, max_sequence_len):
    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))
    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
    return model


# 创建模型
model = create_model(total_words, max_sequence_len)
model.summary()

# 设置checkpoint
checkpoint_path = 'model_checkpoint.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='loss', verbose=1, save_best_only=True, mode='min')

# 训练模型
model.fit(predictors, label, epochs=100, verbose=1, callbacks=[checkpoint])

# 保存tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 45, 100)           900800    
                                                                 
 lstm_10 (LSTM)              (None, 45, 150)           150600    
                                                                 
 lstm_11 (LSTM)              (None, 100)               100400    
                                                                 
 dense_5 (Dense)             (None, 9008)              909808    
                                                                 
Total params: 2,061,608
Trainable params: 2,061,608
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
786/786 [==============================] - ETA: 0s - loss: 8.9245 - accuracy: 0.0640
Epoch 1: loss improved from inf to 8.92454, saving model to model_checkpoint.h5

786/786 [==============================] - 52s 66ms/step - loss: 1.5880 - accuracy: 0.6084
Epoch 28/100
786/786 [==============================] - ETA: 0s - loss: 1.5219 - accuracy: 0.6196
Epoch 28: loss improved from 1.58796 to 1.52191, saving model to model_checkpoint.h5
786/786 [==============================] - 54s 69ms/step - loss: 1.5219 - accuracy: 0.6196
Epoch 29/100
786/786 [==============================] - ETA: 0s - loss: 1.4699 - accuracy: 0.6270
Epoch 29: loss improved from 1.52191 to 1.46992, saving model to model_checkpoint.h5
786/786 [==============================] - 53s 68ms/step - loss: 1.4699 - accuracy: 0.6270
Epoch 30/100
786/786 [==============================] - ETA: 0s - loss: 1.4526 - accuracy: 0.6322
Epoch 30: loss improved from 1.46992 to 1.45258, saving model to model_checkpoint.h5
786/786 [==============================] - 53s 68ms/step - loss: 1.4526 - accuracy: 0.6322
Epoch 31/100
786/786 [==============================] - ETA: 0s - loss: 1.4118 - accura

Epoch 59/100
786/786 [==============================] - ETA: 0s - loss: 0.9752 - accuracy: 0.7324
Epoch 59: loss did not improve from 0.94610
786/786 [==============================] - 53s 67ms/step - loss: 0.9752 - accuracy: 0.7324
Epoch 60/100
786/786 [==============================] - ETA: 0s - loss: 0.9900 - accuracy: 0.7295
Epoch 60: loss did not improve from 0.94610
786/786 [==============================] - 54s 68ms/step - loss: 0.9900 - accuracy: 0.7295
Epoch 61/100
786/786 [==============================] - ETA: 0s - loss: 0.9961 - accuracy: 0.7263
Epoch 61: loss did not improve from 0.94610
786/786 [==============================] - 54s 68ms/step - loss: 0.9961 - accuracy: 0.7263
Epoch 62/100
786/786 [==============================] - ETA: 0s - loss: 1.0424 - accuracy: 0.7144
Epoch 62: loss did not improve from 0.94610
786/786 [==============================] - 53s 68ms/step - loss: 1.0424 - accuracy: 0.7144
Epoch 63/100
786/786 [==============================] - ETA: 0s - lo

# Model V3

In [13]:
# 定义模型
def create_model(total_words, max_sequence_len):
    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))
    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.05), metrics=['accuracy'])
    return model


# 创建模型
model = create_model(total_words, max_sequence_len)
model.summary()

# 设置checkpoint
checkpoint_path = 'model_checkpoint.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='loss', verbose=1, save_best_only=True, mode='min')

# 训练模型
model.fit(predictors, label, epochs=100, verbose=1, callbacks=[checkpoint])

# 保存tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 30, 100)           650300    
                                                                 
 lstm_4 (LSTM)               (None, 30, 150)           150600    
                                                                 
 lstm_5 (LSTM)               (None, 100)               100400    
                                                                 
 dense_2 (Dense)             (None, 6503)              656803    
                                                                 
Total params: 1,558,103
Trainable params: 1,558,103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
944/944 [==============================] - ETA: 0s - loss: 8.0879 - accuracy: 0.0706
Epoch 1: loss improved from inf to 8.08791, saving model to model_checkpoint.h5

944/944 [==============================] - 40s 42ms/step - loss: 2.6019 - accuracy: 0.4105
Epoch 28/100
944/944 [==============================] - ETA: 0s - loss: 2.5502 - accuracy: 0.4170
Epoch 28: loss improved from 2.60189 to 2.55015, saving model to model_checkpoint.h5
944/944 [==============================] - 39s 41ms/step - loss: 2.5502 - accuracy: 0.4170
Epoch 29/100
944/944 [==============================] - ETA: 0s - loss: 2.5125 - accuracy: 0.4228
Epoch 29: loss improved from 2.55015 to 2.51246, saving model to model_checkpoint.h5
944/944 [==============================] - 39s 41ms/step - loss: 2.5125 - accuracy: 0.4228
Epoch 30/100
943/944 [============================>.] - ETA: 0s - loss: 2.4577 - accuracy: 0.4304
Epoch 30: loss improved from 2.51246 to 2.45793, saving model to model_checkpoint.h5
944/944 [==============================] - 39s 41ms/step - loss: 2.4579 - accuracy: 0.4303
Epoch 31/100
943/944 [============================>.] - ETA: 0s - loss: 2.4161 - accura

Epoch 59/100
944/944 [==============================] - ETA: 0s - loss: 1.8692 - accuracy: 0.5350
Epoch 59: loss did not improve from 1.85691
944/944 [==============================] - 46s 48ms/step - loss: 1.8692 - accuracy: 0.5350
Epoch 60/100
944/944 [==============================] - ETA: 0s - loss: 1.8837 - accuracy: 0.5272
Epoch 60: loss did not improve from 1.85691
944/944 [==============================] - 43s 45ms/step - loss: 1.8837 - accuracy: 0.5272
Epoch 61/100
944/944 [==============================] - ETA: 0s - loss: 1.8202 - accuracy: 0.5440
Epoch 61: loss improved from 1.85691 to 1.82021, saving model to model_checkpoint.h5
944/944 [==============================] - 44s 47ms/step - loss: 1.8202 - accuracy: 0.5440
Epoch 62/100
943/944 [============================>.] - ETA: 0s - loss: 1.9371 - accuracy: 0.5287
Epoch 62: loss did not improve from 1.82021
944/944 [==============================] - 43s 45ms/step - loss: 1.9382 - accuracy: 0.5287
Epoch 63/100
944/944 [=====

Epoch 93/100
944/944 [==============================] - ETA: 0s - loss: 1.8144 - accuracy: 0.5503
Epoch 93: loss did not improve from 1.75983
944/944 [==============================] - 40s 42ms/step - loss: 1.8144 - accuracy: 0.5503
Epoch 94/100
943/944 [============================>.] - ETA: 0s - loss: 1.7982 - accuracy: 0.5521
Epoch 94: loss did not improve from 1.75983
944/944 [==============================] - 39s 42ms/step - loss: 1.7982 - accuracy: 0.5521
Epoch 95/100
943/944 [============================>.] - ETA: 0s - loss: 1.7882 - accuracy: 0.5501
Epoch 95: loss did not improve from 1.75983
944/944 [==============================] - 41s 44ms/step - loss: 1.7878 - accuracy: 0.5502
Epoch 96/100
944/944 [==============================] - ETA: 0s - loss: 1.8034 - accuracy: 0.5488
Epoch 96: loss did not improve from 1.75983
944/944 [==============================] - 40s 42ms/step - loss: 1.8034 - accuracy: 0.5488
Epoch 97/100
943/944 [============================>.] - ETA: 0s - lo

In [10]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(titles)
# total_words = len(tokenizer.word_index) + 1

# input_sequences = []
# for line in titles:
#     token_list = tokenizer.texts_to_sequences([line])[0]
#     for i in range(1, len(token_list)):
#         n_gram_sequence = token_list[:i+1]
#         input_sequences.append(n_gram_sequence)

# # 填充序列以保持统一长度
# max_sequence_len = max([len(x) for x in input_sequences])
# input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# # 分割为特征和标签
# predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
# label = to_categorical(label, num_classes=total_words)

# # 定义模型
# def create_model(total_words, max_sequence_len):
#     model = Sequential()
#     model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
#     model.add(LSTM(150, return_sequences=True))
#     model.add(LSTM(100))
#     model.add(Dense(total_words, activation='softmax'))
#     # 编译模型
#     model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
#     return model

# # 创建模型
# model = create_model(total_words, max_sequence_len)
# model.summary()

# # 设置checkpoint
# checkpoint_path = 'model_checkpoint2.h5'
# checkpoint = ModelCheckpoint(checkpoint_path, monitor='loss', verbose=1, save_best_only=True, mode='min')

# # 训练模型
# model.fit(predictors, label, epochs=100, verbose=1, callbacks=[checkpoint])

# dump(tokenizer, open('tokenizer2.pkl', 'wb'))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 45, 100)           900800    
                                                                 
 lstm_6 (LSTM)               (None, 45, 150)           150600    
                                                                 
 lstm_7 (LSTM)               (None, 100)               100400    
                                                                 
 dense_3 (Dense)             (None, 9008)              909808    
                                                                 
Total params: 2,061,608
Trainable params: 2,061,608
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


KeyboardInterrupt: 

In [14]:
# 测试模型：
# 加载模型
model = load_model('model_checkpoint.h5')

# 加载tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

In [15]:
def generate_text(seed_text, next_words, model, tokenizer, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predictions = model.predict(token_list, verbose=0)
        predicted = np.argmax(predictions, axis=-1)[0]
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [16]:
seed_text = "Deep learning"
next_words = 10  # 生成标题的长度

generated_title = generate_text(seed_text, next_words, model, tokenizer, max_sequence_len)
print(generated_title)

Deep learning for mortgage risk matrices route to recover magnetite from bovine


In [26]:
seed_text = "Deep learning"
next_words = 5  # 生成标题的长度

generated_title = generate_text(seed_text, next_words, model, tokenizer, max_sequence_len)
print(generated_title)

Deep learning or refreshment l koch 18881976
